In [ ]:
#| default_exp experiments.merlion.model

In [ ]:
#| export

from typing import Optional
from pathlib import Path
from os.path import isdir

import merlion.models.forecast
from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig

from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample

from merlion.evaluate.forecast import ForecastMetric
from merlion.models.ensemble.combine import Mean, ModelSelector
from merlion.models.ensemble.forecast import ForecasterEnsemble, ForecasterEnsembleConfig

# from national.data_preprocessing.file_system import FileHandler
from national.util import constants


from IPython.display import display, Markdown

Submissions are evaluated on the Root Mean Square Percentage Error (RMSPE). The RMSPE is calculated as



$$\operatorname{RMSPE}=\sqrt{\frac{1}{T}\sum_{t=1}^T\left(\frac{ \hat y_t - y_t}{y_t}\right)^2}$$

where $y_t$ denotes the sales of a single store on a single day and $\hat y_t$ denotes the corresponding prediction. Any day and store with 0 sales is ignored in scoring.

In [ ]:
#| export
MerlionModel: type = type(merlion.models.forecast)
MerlionConfig: type = type(ArimaConfig)

In [ ]:
#| export


class Evaluation:

    def __init_(self):
        self.inference: Optional[float] = None
        self.train: Optional[float] = None


class Results:

    def __init__(self):
        self.train: Optional[float] = None
        self.val: Optional[float] = None
        self.test: Optional[float] = None
        self.future: Optional[float] = None

In [ ]:
#| export


class Model:
    """All models are initialized using the syntax ModelClass(config),
where config is a model-specific configuration object.
This is where you specify any algorithm-specific hyperparameters,
as well as any data pre-processing transforms.
    """
    def __init__(
        self,
        config: MerlionConfig,
        model_f: MerlionModel,
        load_model: bool,
        name: str=None,
        **args,
    ):
        self.config = config
        self.model_f = model_f

        self.forecast = Results()
        self.stderr = Results()
        self.evaluation = Evaluation()

        _model_dir = "/workspaces/niloIQ/models/merlion"

        self.model = model_f(
            config=config,
            **args,
        )

        self.name = type(self.model).__name__ if name is None else name

        self.model_path = Path(_model_dir, self.name)

        self.load_model = load_model and isdir(self.model_path)

        if self.load_model:
            #print(self.name)
            self.model = model_f.load(self.model_path, )

In [ ]:
#| export


class Models:
    """
        ARIMA assumes that input data is sampled at a regular interval,
so we set its transform to resample at that interval.
We must also specify a maximum prediction horizon.

MSES assumes that the input data is sampled at a regular interval,
and requires us to specify a maximum prediction horizon. We will
also specify its look-back hyperparameter to be 60 here
"""
    def __init__(
        self,
        granularity,
        load_models: bool = False,
    ):

        self.granularity = granularity

        self.arima = Model(
            config=ArimaConfig(
                max_forecast_steps=constants.ARIMA_MAX_STEPS,
                order=(20, 1, 5),
                transform=TemporalResample(granularity=granularity),
            ),
            model_f=Arima,
            load_model=load_models,
        )

        self.prophet = Model(
            config=ProphetConfig(
                max_forecast_steps=None,
                transform=Identity(),
            ),
            model_f=Prophet,
            load_model=load_models,
        )

        self.mses = Model(
            config=MSESConfig(
                max_forecast_steps=constants.SMES_MAX_STEPS,
                max_backstep=60,
                transform=TemporalResample(granularity=granularity),
            ),
            model_f=MSES,
            load_model=load_models,
        )

        self.model_names = [
            type(self.arima.model).__name__,
            type(self.prophet.model).__name__,
            type(self.mses).__name__,
        ]

        # self.ensemble = Model(
        #     config=ForecasterEnsembleConfig(
        #         combiner=Mean(),
        #         model_configs=[
        #             (type(self.arima.model).__name__, self.arima.config),
        #             (type(self.prophet.model).__name__, self.prophet.config),
        #             (type(self.mses.model).__name__, self.mses.config),
        #         ]
        #     ),
        #     model_f=ForecasterEnsemble,
        #     load_model=load_models,
        # )

        # self.selector = Model(
        #     config=ForecasterEnsembleConfig(
        #         combiner=ModelSelector(metric=ForecastMetric.sMAPE),
        #     ),
        #     model_f=ForecasterEnsemble,
        #     models=[
        #         self.arima.model,
        #         self.prophet.model,
        #         self.mses.model,
        #     ],
        #     load_model=load_models,
        #     name= "Selector",
        # )

        # self.partial_ensemble = Model(
        #     config=ForecasterEnsembleConfig(
        #         combiner=Mean(),
        #         model_configs=[
        #             (type(self.arima.model).__name__, self.arima.config),
        #             (type(self.prophet.model).__name__, self.prophet.config),
        #         ]
        #     ),
        #     model_f=ForecasterEnsemble,
        #     load_model=load_models,
        #     name= "PartialEnsemble",
        # )

In [ ]:
from national.experiments.merlion import model as merlion_model
models = merlion_model.Models(granularity='W-MON')

##%% export

The models considered are ARIMA, Prophet and SMES. Also an ensemble of these three models has been considered

In [ ]:
#from national.experiments.merlion import model as merlion_model

loaded_models =Models(granularity='W-MON')